# Loading, Joining and Writing Out Data with AWS Glue  
This sample shows how to manipulate data files on  AWS S3 from Amazon SageMaker Notebook (Pyspark)

In [1]:
# create glue context first
glueContext = GlueContext(SparkContext.getOrCreate())

NameError: name 'GlueContext' is not defined

In [ ]:
# creating dataframes from existing athena catelog  
up_features = glueContext.create_dynamic_frame_from_options(connection_type = "parquet", connection_options = {"paths": ["s3://imba-shawn/features/up_feature_db/"]})
prd_features = glueContext.create_dynamic_frame_from_options(connection_type = "parquet", connection_options = {"paths": ["s3://imba-shawn/features/prd_feature_db/"]})
user_features_1 = glueContext.create_dynamic_frame_from_options(connection_type = "parquet", connection_options = {"paths": ["s3://imba-shawn/features/user_features_1_db/"]})
user_features_2 = glueContext.create_dynamic_frame_from_options(connection_type = "parquet", connection_options = {"paths": ["s3://imba-shawn/features/user_features_2_db/"]})

In [ ]:
# join user features together
users = Join.apply(user_features_1.rename_field('user_id','user_id1'), user_features_2, 'user_id1', 'user_id').drop_fields(['user_id1'])

In [ ]:
# join everything together
df = Join.apply(Join.apply(up_features, 
                    users.rename_field('user_id','user_id1'), 
                    'user_id','user_id1').drop_fields(['user_id1']),
        prd_features.rename_field('product_id','product_id1'), 
        'product_id','product_id1').drop_fields(['product_id1'])

In [ ]:
# convert glue dynamic dataframe to spark dataframe
df_spark = df.toDF()
df_spark.repartition(1).write.mode('overwrite').format('csv').save("s3://imba-shawn/output", header = 'true')